In [ ]:
from IPython.display import clear_output
!pip install albumentations==0.4.6


In [2]:
# ALL IMPORTS 
import cv2 
import pandas as pd
import numpy as np
import os
from PIL import  Image
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torch 
import torchvision
from torch.utils.data import DataLoader,Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as pyplot

In [ ]:
#bring dataset home ... :P
!gdown --id  1o36HQn1Q8m7s6w30ik3hD2W0Qz2iHTV9
!unrar x augmented.rar
clear_output()
!mkdir dataset
!mv '/content/Final Train Dataset' '/content/dataset/train' 
!mv '/content/test' '/content/dataset/test'
 

mv: cannot stat '/content/Final Train Dataset': No such file or directory
mv: cannot stat '/content/test': No such file or directory


In [ ]:
#remove useless images 
corrupt_files = ['231.jpg', '231.xml', 'Pias (359).PNG','Pias (359).xml', 'Pias (360).PNG', 'Pias (360).xml']

%cd /content/dataset/train/

for file in corrupt_files:
    file_path = os.path.join('/content/dataset/train/', file)  
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f'{file} is removed successfully')
    else:
        print(f'{file} is already deleted')

%cd /content/

In [ ]:
import glob
# Reading Image file paths
formats = ['jpg', 'jpeg', 'JPG', 'png','PNG']
image_file_list = []
for format in formats:
    image_file_list.extend(glob.glob(f'/content/dataset/train/*.{format}'))

# Reading XML label file paths
label_file_list_xml = glob.glob('/content/dataset/train/*.xml')

print(f'Image files found: {len(image_file_list)} \nLabel files found: { len(label_file_list_xml)}')

In [ ]:
#Trainning dataframe
dataframe=pd.DataFrame(columns=['id','class','xmax','ymax','xmin','ymin','height','width'])

In [ ]:
from xml.dom import minidom
#label mapping 
lut={"ambulance": 0,
     "auto rickshaw": 1,
     "bicycle": 2,
     "bus": 3,
     "car": 4,
     "garbagevan": 5,
     "human hauler": 6,
     "minibus": 7,
     "minivan": 8,
     "motorbike": 9,
     "pickup": 10,
     "army vehicle": 11,
     "policecar": 12,
     "rickshaw": 13,
     "scooter": 14,
     "suv": 15,
     "taxi": 16,
     "three wheelers (CNG)": 17,
     "truck": 18,
     "van": 19,
     "wheelbarrow": 20
     }


label_count ={}

print(f'Object Names: {list(lut.keys())}' )


def convert_xml2csv(filelist,lut):
    """
    filelist: list of .xml file paths to convert to .txt file
    lut: a dictionary containing class_name to class_index mapping
    """
    #Trainning dataframe
    normal=pd.DataFrame(columns=['id','class','xmin','ymin','xmax','ymax','height','width'])
    dataframe=pd.DataFrame(columns=['id','xmin','ymin','xmax','ymax','class'])
    for fname in filelist:
        xmldoc = minidom.parse(fname)
        fname_out = (fname[:-4]+'.txt')

        with open(fname_out, "w") as f:
            # print(f'processing{fname}')
            image_name=(xmldoc.getElementsByTagName('filename')[0]).firstChild.data
            if(os.path.isfile('/content/dataset/train/'+image_name)==False):
              fmt=image_name.split('.')[-1]
              image_name=fname.split('/')[-1]
              image_name=image_name.split('.xml')[0]+'.'+fmt
              print(image_name+'           ------->   '+ fname)
            itemlist = xmldoc.getElementsByTagName('object')
            size = xmldoc.getElementsByTagName('size')[0]
            width = int((size.getElementsByTagName('width')[0]).firstChild.data)
            height = int((size.getElementsByTagName('height')[0]).firstChild.data)

            for item in itemlist:
                # get class label
                classid =  (item.getElementsByTagName('name')[0]).firstChild.data
                if classid in lut:
                    label_str = str(lut[classid])
                else:
                    label_str = "-1"
                    print ("warning: label '%s' not in look-up table for file '%s'" % classid, fname )
                # get bbox coordinates
                xmin = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('xmin')[0]).firstChild.data
                ymin = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('ymin')[0]).firstChild.data
                xmax = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('xmax')[0]).firstChild.data
                ymax = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('ymax')[0]).firstChild.data
                xmin,ymin,xmax,ymax=(float(xmin)/width)*1024, (float(ymin)/height)*1024, (float(xmax)/width)*1024, (float(ymax)/height)*1024
                dataframe=dataframe.append({'id':'/content/dataset/train/'+image_name.split('/')[-1],'xmin':int(xmin),'ymin':int(ymin),'xmax':int(xmax),'ymax':int(ymax),'class':classid},ignore_index=True)
                #dataframe=normal.append({'id':image_name.split('/')[-1],'class':classid,'xmin':int(xmin),'ymin':int(ymin),'xmax':int(xmax),'ymax':int(ymax),'height':1024,'width':1024},ignore_index=True)
                label_count[classid] = label_count.get(classid, 0) + 1

               # f.write(label_str + " " + " ".join([("%.6f" % a) for a in bb]) + '\n')
        # print ("wrote %s" % fname_out)
    return dataframe


Object Names: ['ambulance', 'auto rickshaw', 'bicycle', 'bus', 'car', 'garbagevan', 'human hauler', 'minibus', 'minivan', 'motorbike', 'pickup', 'army vehicle', 'policecar', 'rickshaw', 'scooter', 'suv', 'taxi', 'three wheelers (CNG)', 'truck', 'van', 'wheelbarrow']


In [ ]:
def resize_images(file_list, width = 1024, height = 1024, overwrite = True, save_dir = ''):
    total_files = len(file_list)
    idx = 1
    for path in file_list:
        img = Image.open(path)
        img_resized = img.resize((width, height))
        if overwrite:
            img_resized.save(path)
            filename = path.split('/')[-1] 
            print(f"{idx}/{total_files}: {filename} {img.size}--> ({width}x{height})")
        else:
            filename = path.split('/')[-1]
            img_resized.save(save_dir + filename)
            print(f'{filename} saved to {save_dir}')
        idx +=1


In [ ]:
#resize_images(image_file_list,overwrite=True)
training_dataframe = convert_xml2csv(label_file_list_xml,lut)


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(training_dataframe['id'].unique(), test_size=0.25)
traindf=pd.DataFrame()
testdf=pd.DataFrame()
for i in range(train.size):
  traindf=traindf.append(training_dataframe[training_dataframe['id']==train[i]])
for i in range(test.size):
  testdf=testdf.append(training_dataframe[training_dataframe['id']==test[i]])

In [ ]:
testdf.sort_index(inplace=True)
testdf.reset_index(inplace=True,drop=True)
traindf.sort_index(inplace=True)
traindf.reset_index(inplace=True,drop=True)

print(str(traindf['id'].unique().shape)+'   '+str(testdf['id'].unique().shape))

(2850,)   (950,)


In [ ]:
#training_dataframe.to_csv('train.csv')
traindf.to_csv('/content/dataset/train.csv',header=None,index=False)
testdf.to_csv('/content/dataset/test.csv',header=None,index=False)
training_dataframe.to_csv('/content/dataset/whole.csv',header=None,index=False)

In [ ]:
resize_images(image_file_list,overwrite=True)

In [ ]:
%cd /content
!zip -r retinadata.zip dataset

In [ ]:
!mv retinadata.zip '/content/drive/MyDrive/Colab Notebooks/Dhaka ai/'

In [ ]:
classInstances=training_dataframe['class'].value_counts().to_frame()

classInstances

In [ ]:
import pandas as pd
import plotly.express as px

# DataFrame Generatio
classInstances=training_dataframe['class'].value_counts()
classInstances.columns=['class','count']
# Plotting
fig = px.bar(classInstances, x=classInstances.keys(), y='class',  color='class',
    orientation='v', 
    title='Frequency of the Labels in Dhaka.ai-2020 Challenge', 
    color_continuous_scale=px.colors.sequential.Viridis_r
)
fig.update_layout(title_x=0.5, xaxis_title = 'Labels', yaxis_title = 'Label Count')
fig.update_xaxes(tickangle=60)
fig.show()


anotherfig=px.pie(classInstances,values='class',names=classInstances.keys(),title="PIE chart of class distribution")
anotherfig.update_layout(title_x=.5,  margin=dict(l=20, r=20, t=20, b=20))

anotherfig.show()

# TRAINING SECTION 

I'll use faster rcnn for 1st phase tranning. After this esemble techniques will be used

### <font color='red'> how to deal with class imbalance issuse??? 
 * Focal loss
 * label smoothing
 * bi-linear focal loss 




In [3]:
!rm -r dataset
#!gdown --id 1iPNvdTbCFXrMOuz2xxdEl4rTavQzTGbr
!gdown --id 1cFkcNkacUov67S88ZHkDQgTDqoxwbbXF   #retina net modified
!unzip retinadata.zip 
'''
  dataset
   -train 
       -img.jpg
       -img.xml
       -img2.jpg
       -img2.xml
        .......
   -test
       -testimg.jpg
       -testimg.xml
       -testimg.jpg
       ......... 
   -.....
   -......
'''
clear_output()

In [ ]:
#trainCsv.to_csv('train.csv', sep='\t')

trainCsv = pd.read_csv("/content/dataset/train.csv",index_col=0)
trainCsv.head()

In [ ]:
trainCsv.drop(columns='Unnamed: 0',axis=1,inplace=True)
trainCsv[trainCsv['id']=='01.jpg']

In [ ]:
#label mapping 
import csv
lut={"ambulance": 0,
     "auto rickshaw": 1,
     "bicycle": 2,
     "bus": 3,
     "car": 4,
     "garbagevan": 5,
     "human hauler": 6,
     "minibus": 7,
     "minivan": 8,
     "motorbike": 9,
     "pickup": 10,
     "army vehicle": 11,
     "policecar": 12,
     "rickshaw": 13,
     "scooter": 14,
     "suv": 15,
     "taxi": 16,
     "three wheelers (CNG)": 17,
     "truck": 18,
     "van": 19,
     "wheelbarrow": 20
     }

for key in lut.keys():
  lut[key]+=1
labeldf = pd.DataFrame({'class_name':lut.keys(),'id':lut.values()})

labeldf.id=labeldf.id.astype('int32')
labeldf.class_name=labeldf.class_name.astype('str')
labeldf.to_csv('/content/dataset/labelmap.csv',mode = 'w',header=None,index=False)
labeldf=pd.read_csv('/content/dataset/labelmap.csv')
train=pd.read_csv('/content/dataset/train.csv')
test=pd.read_csv('/content/dataset/test.csv')


'''
labeldf.to_csv('/content/dataset/labelmap.csv',mode = 'w',header=None,index=False)
train.to_csv('/content/dataset/train.csv',mode = 'w',header=None,index=False)
test.to_csv('/content/dataset/test.csv',mode = 'w',header=None,index=False)
'''
test.head()


In [4]:
!git clone https://github.com/ViswanathaReddyGajjala/EfficientNet-RetinaNet.git

Cloning into 'EfficientNet-RetinaNet'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 412 (delta 77), reused 0 (delta 0), pack-reused 280
Receiving objects: 100% (412/412), 714.29 KiB | 22.32 MiB/s, done.
Resolving deltas: 100% (221/221), done.


In [ ]:
!zip retinanetcode.zip /content/EfficientNet-RetinaNet

	zip warning: name not matched: /content/EfficientNet-RetinaNet

zip error: Nothing to do! (retinanetcode.zip)


In [6]:
%cd /content/EfficientNet-RetinaNet/
!gdown --id 1UkxVl0cIQes-_r37wN7p8bfBjl5qrJ5N
clear_output()

In [ ]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./
clear_output()

# New Section


```js
function ClickConnect(){

console.log("Working"); 
document.querySelector("#comments > span").click() 
}
setInterval(ClickConnect,5000)

```



In [ ]:
%cd /content/EfficientNet-RetinaNet/
!python train.py "/content/dataset/train.csv" "/content/dataset/test.csv" "/content/dataset/labelmap.csv" "b4" 40 4


mAP:
ambulance: 0.24196698566666672
auto rickshaw: 0.017475728155339806
bicycle: 0.11605524727976435
bus: 0.12016138453847142
car: 0.10155344910020636
garbagevan: 0.46143146327683754
human hauler: 0.0315918743651216
minibus: 0.05102040816326531
minivan: 0.0
motorbike: 0.08441496980762533
pickup: 0.04634773902571053
army vehicle: 0.48264544061162296
policecar: 0.426572115043262
rickshaw: 0.08754920820307988
scooter: 0.3401680672268907
suv: 0.012565692109777347
taxi: 0.2739296115333205
three wheelers (CNG): 0.09358210332057251
truck: 0.09732470514495892
van: 0.013804927739430255
wheelbarrow: 0.3946572013844742
Epoch: 5 | Iteration: 299 | Classification loss: 0.28416 |                       Regression loss: 0.40977 | Running loss: 0.79001
Epoch: 5 | Iteration: 599 | Classification loss: 0.25031 |                       Regression loss: 0.44106 | Running loss: 0.76925

mAP:
ambulance: 0.3635363623031394
auto rickshaw: 0.024993516231972306
bicycle: 0.14196620830237
bus: 0.12249349025070885


In [ ]:
%cd /content/
!rm  *.png

/content
rm: cannot remove '*.png': No such file or directory


In [ ]:
from __future__ import print_function, division
import torch
import numpy as np
import skimage.io
import skimage.transform
import skimage.color
import skimage
from dataloader import UnNormalizer

import time
import cv2
import argparse
import  torchvision 

!rm -r output
!mkdir output
class Resize_Img():
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, img, min_side=1024, max_side=1024):

        image = img
        '''  
        rows, cols, cns = image.shape

        smallest_side = min(rows, cols)

        # rescale the image so the smallest side is min_side
        scale = min_side / smallest_side

        # check if the largest side is now greater than max_side, which can happen
        # when images have a large aspect ratio
        largest_side = max(rows, cols)

        if largest_side * scale > max_side:
            scale = max_side / largest_side
    
        # resize the image with the computed scale
        image = skimage.transform.resize(image, (int(round(rows*scale)), int(round((cols*scale)))))
        '''
        rows, cols, cns = image.shape


        new_image = np.zeros((rows , cols, cns)).astype(np.float32)
        new_image[:rows, :cols, :] = image.astype(np.float32)

        
        scale=1
        return {'img': torch.from_numpy(new_image), 'scale': scale}


def collat(data):
    imgs = data
    widths = [int(s.shape[0]) for s in imgs]
    heights = [int(s.shape[1]) for s in imgs]
    batch_size = 1

    # print (batch_size)
    max_width = np.array(widths).max()
    max_height = np.array(heights).max()

    padded_imgs = torch.zeros(batch_size, max_width, max_height, 3)

    for i in range(batch_size):
        img = imgs[i]
        img = torch.Tensor(img)
        padded_imgs[i, :int(img.shape[0]), :int(img.shape[1]), :] = img

    padded_imgs = padded_imgs.permute(0, 3, 1, 2)

    return padded_imgs


def draw_caption(image, box, caption):
    b = np.array(box).astype(int)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)

import numpy as np
def nms(boxes, overlapThresh):

	if len(boxes) == 0:
		return []
	
	if boxes.dtype.kind == "i":
		boxes = boxes.astype("float")	 
	pick = []
	x1 = boxes[:,2]
	y1 = boxes[:,3]
	x2 = boxes[:,4]
	y2 = boxes[:,5]
	area = (x2 - x1 + 1) * (y2 - y1 + 1)
	idxs = np.argsort(d[:,1])
	
	while len(idxs) > 0:
		
		last = len(idxs) - 1
		i = idxs[last]
		pick.append(i)
		
		xx1 = np.maximum(x1[i], x1[idxs[:last]])
		yy1 = np.maximum(y1[i], y1[idxs[:last]])
		xx2 = np.minimum(x2[i], x2[idxs[:last]])
		yy2 = np.minimum(y2[i], y2[idxs[:last]])
		
		w = np.maximum(0, xx2 - xx1 + 1)
		h = np.maximum(0, yy2 - yy1 + 1)
		
		overlap = (w * h) / area[idxs[:last]]
		
		idxs = np.delete(idxs, np.concatenate(([last],
			np.where(overlap > overlapThresh and boxes[:last,1]==boxes[last,1])[0])))
	
	return boxes[pick].astype("float32")

def visualize(model_path,image_path):
    retinanet = torch.load(model_path)
    
    custom_labels ={
     "ambulance": 0,
     "auto rickshaw": 1,
     "bicycle": 2,
     "bus": 3,
     "car": 4,
     "garbagevan": 5,
     "human hauler": 6,
     "minibus": 7,
     "minivan": 8,
     "motorbike": 9,
     "pickup": 10,
     "army vehicle": 11,
     "policecar": 12,
     "rickshaw": 13,
     "scooter": 14,
     "suv": 15,
     "taxi": 16,
     "three wheelers (CNG)": 17,
     "truck": 18,
     "van": 19,
     "wheelbarrow": 20
     }

    label_map = {k: v + 1 for v, k in enumerate(custom_labels)}
    label_map['background'] = 0
    rev_label_map = {v: k for k, v in label_map.items()} # Inverse mapping
    use_gpu=True
    if use_gpu:
        retinanet = retinanet.cuda()
      
    unnormalize = UnNormalizer()
    retinanet.eval()
    
    submission_dataframe=pd.DataFrame(columns=['image_id','class','score','xmin','ymin','xmax','ymax','width','height'])
    with torch.no_grad():
      for img_path in image_path:
        

        st = time.time();
        img = cv2.imread(img_path)
        img =cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32)/255.0
        
        '''
        mean = np.array([[[0.485, 0.456, 0.406]]])
        std = np.array([[[0.229, 0.224, 0.225]]])
        img = ((img.astype(np.float32)-mean)/std)
        '''
        image_resizer = Resize_Img()
        img = image_resizer(img)['img']

        img = np.expand_dims(img, axis=0)
        img = collat(img)
        
        scores, classification, transformed_anchors = retinanet(img.cuda().float())
        print('Elapsed time: {}'.format(time.time()-st))
        idxs = np.where(scores.cpu()>0.35)
        #img=255*img
        img = np.array(255 * img[0, :, :, :]).copy()

        img[img<0] = 0
        img[img>255] = 255

        img = np.transpose(img, (1, 2, 0))
        
        img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_BGR2RGB)
        detections=[]
        torchdets=[]
        sc=[]
        cls=[]
        for j in range(idxs[0].shape[0]):
            bbox = transformed_anchors[idxs[0][j], :]
            x1 = int(bbox[0])
            y1 = int(bbox[1])
            x2 = int(bbox[2])
            y2 = int(bbox[3])
            label_name = rev_label_map[int(classification[idxs[0][j]])]
            
      
            x=scores[j].item()
            detections.append([int(classification[idxs[0][j]]),x,x1,y1,x2,y2,1024,1024])

            torchdets .append([x1,y1,x2,y2])
            sc.append(x)
            cls.append(int(classification[idxs[0][j]]))
          

        #det=np.array(detections)
        #det=nms(det,.55)
        
        cls=np.array(cls,dtype=np.int32)
        det=torch.FloatTensor(torchdets)
        sc=torch.FloatTensor(sc)
        idx=torchvision.ops.nms(det,sc,.55)
        det,sc,cls=det[idx].numpy(),sc[idx].numpy(),cls[idx]
        det=det.astype("float32")
        id=img_path.split('/')[-1]
        df=pd.DataFrame(columns=['image_id','class','score','xmin','ymin','xmax','ymax','width','height'])
        print(id)
        omni=np.insert(det,4,sc,axis=1)
        omni=np.insert(omni,5,cls,axis=1)     
        #print(omni)
        for d in omni:

          df=df.append({'image_id':id,'class':rev_label_map[int(d[5])],'score':d[4],'xmin':d[0],'ymin':d[1],'xmax':d[2],'ymax':d[3],'width':1024,'height':1024},ignore_index=True)
          print('                          '+rev_label_map[int(d[5])]+'   score:'+str(d[4])+'      pos:' +str(d[0:4])) 
          
          caption=str(rev_label_map[int(d[5])])+' '+str('%.2f'%d[4])     
          cv2.rectangle(img, (d[0], d[1]), (d[2], d[3]),color=(0, 0, 255), thickness=2)
          cv2.putText(img, caption, (int(d[0]), int(d[1]) - 10), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 3)

          '''
          df=df.append({'image_id':id,'class':rev_label_map[int(d[0])],'score':d[1],'xmin':d[2],'ymin':d[3],'xmax':d[4],'ymax':d[5],'width':1024,'height':1024},ignore_index=True)
          print('                          '+rev_label_map[int(d[0])]+'   score:'+str(d[1])+'      pos:' +str(d[2:6]) ) 
          caption=str(rev_label_map[int(d[0])])+' '+"%.2f"%d[1]       
          cv2.rectangle(img, (d[2], d[3]), (d[4], d[5]), color=(0, 0, 255), thickness=2)
          cv2.putText(img, caption, (int(d[2]), int(d[3]) - 10), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 1)
          '''

        submission_dataframe=submission_dataframe.append(df,ignore_index=True)
        
        path='/content/output/'+id +'.png'
        cv2.imwrite(path, img)
        
    return submission_dataframe
      

In [ ]:
!cp ../dataset/labelmap.csv ./label.csv
!python inference.py "/content/EfficientNetb4_retinanet_17_map0.6199770760597492.pt" "/content/dataset/test/Asraf_50_jpg.rf.7026694f0b9f37a6790982295c7e8663.jpg" True

In [ ]:
%cd /content
from glob import glob 
formats = ['jpg', 'jpeg', 'JPG', 'png','PNG']

image_path=[]
for f in formats:
  image_path.extend(glob(f'/content/dataset/test/*.{f}'))

model_path='/content/EfficientNet-RetinaNet/EfficientNetb4_retinanet_17_map0.6199770760597492.pt'
len(image_path)


/content


500

In [ ]:

submit=visualize(model_path,image_path)
submit.to_csv('/content/submission.csv',index=False)
#submit.sort_values(by=['image_id'])
submit.head()

In [ ]:
import glob
from IPython.display import  display
from PIL import Image
for imageName in glob.glob('/content/output/*.png'):
    print(imageName)
    print("\n")
    img=cv2.imread(imageName)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(480,600))
    img=np.array(img)
    display(Image.fromarray(img))
    print("\n")